# EDA & Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
import random 
import math as m
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus                          import wordnet as wnet 
from nltk.stem.wordnet                    import WordNetLemmatizer
from nltk                                 import word_tokenize, pos_tag

### Loading DataSets

In [ ]:
data_proj = pd.read_csv('projects.csv')
data_res  = pd.read_csv('resources.csv')

# #Preprocessing 

# 1. Categorial Features

## a.) project_grade_category

In [ ]:
data_proj['project_grade_category'].value_counts()

>  Replacing all non alphanumeric character with '_'
>  Converting all character to lower

In [ ]:
data_proj['project_grade_category']=data_proj['project_grade_category'].replace(to_replace ='[^0-9a-zA-Z]+' , value = '_' , regex = True)
data_proj['project_grade_category']=data_proj['project_grade_category'].str.lower()
data_proj['project_grade_category'].value_counts()

## b.) project_subject_categories

In [ ]:
data_proj['project_subject_categories'].head(5)

In [ ]:
data_proj['project_subject_categories']=data_proj['project_subject_categories'].str.lower()
data_proj['project_subject_categories']=data_proj['project_subject_categories'].replace(to_replace =' the ' , value = '' , regex = True)
data_proj['project_subject_categories']=data_proj['project_subject_categories'].replace(to_replace ='[^0-9a-zA-Z]+' , value = ' ' , regex = True)
data_proj['project_subject_categories'].head(5)

## c.) school_state

In [ ]:
data_proj['school_state'].value_counts()

In [ ]:
data_proj['school_state']=data_proj['school_state'].str.lower()
data_proj['school_state'].head(5)

## d.) project_subject_subcategories

In [ ]:
data_proj['project_subject_subcategories'].head(5)

In [ ]:
data_proj['project_subject_subcategories']=data_proj['project_subject_subcategories'].str.lower()
data_proj['project_subject_subcategories']=data_proj['project_subject_subcategories'].replace(to_replace =' the ' , value = '' , regex = True)
data_proj['project_subject_subcategories']=data_proj['project_subject_subcategories'].replace(to_replace ='[^0-9a-zA-Z]+' , value = ' ' , regex = True)
data_proj['project_subject_subcategories'].head(5)

## e.) teacher_prefix

In [ ]:
data_proj['teacher_prefix'].value_counts()

In [ ]:
data_proj['teacher_prefix']=data_proj['teacher_prefix'].str.lower()
data_proj['teacher_prefix']=data_proj['teacher_prefix'].replace(to_replace ='[^0-9a-zA-Z]+' , value = '' , regex = True)
data_proj['teacher_prefix'].value_counts()

## 2.) Documents

In [ ]:
# Modified preprocessing. Added lematization
# fun decontracted,stopword list and fun preprocessing From notebooks provided
# https://stackoverflow.com/a/47091490/4084039
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]



# Combining all the above stundents 
def preprocess_text(text_data):
#     https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word
    word_type = {'N':wnet.NOUN, 'V':wnet.VERB , 'R':wnet.ADV, 'J':wnet.ADJ}
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        sent = ' '.join(WordNetLemmatizer().lemmatize(i,word_type.get(j[0],wnet.NOUN))  for i,j in pos_tag(word_tokenize(sent))   )
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

## a.) project_resource_summary

In [ ]:
data_proj['project_resource_summary'].head(5)

In [ ]:
data_proj['project_resource_summary'] = preprocess_text(data_proj['project_resource_summary'].values)

In [ ]:
data_proj['project_resource_summary'].head(5)

## b.) project essay 
> Merging all four essay column & creating new feature essay

In [ ]:
data_proj['essay'] = data_proj['project_essay_1'] +data_proj['project_essay_2'] \
                    +data_proj['project_essay_3'].fillna('') +data_proj['project_essay_4'].fillna('')

In [ ]:
data_proj['essay'].head(5)

In [ ]:
data_proj['essay'] = preprocess_text(data_proj['essay'].values)
data_proj['essay'].head(5)

## d.) project_title

In [ ]:
data_proj['project_title'].head(5)

In [ ]:
data_proj['project_title'] = preprocess_text(data_proj['project_title'].values)
data_proj['project_title'].head(5)

# 3. Numerical Values

> Aggreting numerical feature of resource data & normalizing it

In [ ]:
data_num = data_res.groupby('id').agg({'price':'sum', 'quantity':'sum'}).reset_index()

>  merging project data & feature data

In [ ]:
data = pd.merge(data_proj,data_num, on='id', how='left')

In [ ]:
data.head(10)

# #Finalizing Data

### Choosing Dataset to use

> Droping few samlpe points

In [ ]:
for i in data.columns:
    print(i,'--->',len(data[data[i].isnull()]))

In [ ]:
print('droping ',len(data)-len(data[data['teacher_prefix'].notnull()]),' points ')
data_f = data[data['teacher_prefix'].notnull()].reset_index(drop=True)
print('Toal data point = ',len(data_f))

> Droping useless columns

In [ ]:
data_f.columns

In [ ]:
data_f=data_f.drop(['Unnamed: 0','digit_in_project_summary','project_essay_1','project_essay_2'\
                 ,'project_essay_3','project_essay_4'],axis=1)
data_f.columns

> Checking is still any value missing except in price and quantity bcoz for price and quantity we will impute it later 

In [ ]:
for i in data_f.columns:
    print(i,'--->',len(data_f[data_f[i].isnull()]))

> Exporting data 

In [ ]:
data_f.to_csv('donor_choose.csv',index=False)